In [7]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("./data/matches_info_SL.csv")

# Split the "Result" column into two new columns
df[['result_end', 'result_ht']] = df['Result'].str.split(expand=True)

# Remove the parentheses from the "result_ht" column
df['result_ht'] = df['result_ht'].str.strip('()')

# Extract goals for home team and away team
df[['goals_HomeTeam', 'goals_AwayTeam']] = df['result_end'].str.split(':', expand=True)

# Save the cleaned data to a new CSV file
df.to_csv("./data/matches_info_SL_cleaned.csv", index=False)

# Display the first few rows of the cleaned data
print(df.head())

   Match ID       Home Team  HomeTeamPoints  AwayTeamPoints  \
0   3840895  BSC Young Boys               3               0   
1   3840896   FC Winterthur               1               1   
2   3840897       FC Lugano               0               3   
3   3840898     Servette FC               3               0   
4   3840899       FC Luzern               1               1   

                 Away Team     Result                      League result_end  \
0                FC Zürich  4:0 (0:0)  Credit Suisse Super League        4:0   
1            FC Basel 1893  1:1 (1:0)  Credit Suisse Super League        1:1   
2                  FC Sion  2:3 (0:1)  Credit Suisse Super League        2:3   
3       FC St. Gallen 1879  1:0 (1:0)  Credit Suisse Super League        1:0   
4  Grasshopper Club Zurich  1:1 (1:1)  Credit Suisse Super League        1:1   

  result_ht goals_HomeTeam goals_AwayTeam  
0       0:0              4              0  
1       1:0              1              1  
2       

In [9]:
# Read the CSV file
df_lineups = pd.read_csv("./data/lineups_2022_2023_SL.csv")

# Split the "Gameday" column to extract matchday, weekday, date, and time
df_lineups[['matchday', 'weekday', 'date', 'time']] = df_lineups['Gameday'].str.split('|').apply(lambda x: pd.Series([x[0].strip(), x[1].strip().split(',')[0].strip(), x[1].strip().split(',')[1].strip(), x[2].strip()]))

# Convert the "date" column to proper datetime format
df_lineups['date'] = pd.to_datetime(df_lineups['date'])

# Save the cleaned data to a new CSV file
df_lineups.to_csv("./data/lineups_2022_2023_SL_cleaned.csv", index=False)

# Display the first few rows of the cleaned data
print(df_lineups.head())

      Position               Player  Age Market Value            Club  \
0   Goalkeeper   David von Ballmoos   27       €2.50m  BSC Young Boys   
1  Centre-Back       Cédric Zesiger   24       €3.20m  BSC Young Boys   
2  Centre-Back  Fabian Lustenberger   34        €400k  BSC Young Boys   
3    Left-Back       Ulisses Garcia   26       €2.00m  BSC Young Boys   
4   Right-Back           Lewin Blum   20        €750k  BSC Young Boys   

                                   Gameday   H/A    Status  Match ID  \
0  1. Matchday | Sat, 7/16/22   |  6:00 PM  Home  Starting   3840895   
1  1. Matchday | Sat, 7/16/22   |  6:00 PM  Home  Starting   3840895   
2  1. Matchday | Sat, 7/16/22   |  6:00 PM  Home  Starting   3840895   
3  1. Matchday | Sat, 7/16/22   |  6:00 PM  Home  Starting   3840895   
4  1. Matchday | Sat, 7/16/22   |  6:00 PM  Home  Starting   3840895   

      matchday weekday       date     time  
0  1. Matchday     Sat 2022-07-16  6:00 PM  
1  1. Matchday     Sat 2022-07-16  6:0

C:\Users\moren\AppData\Local\Temp\ipykernel_16968\4256795980.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_lineups['date'] = pd.to_datetime(df_lineups['date'])


In [21]:
# Read the CSV file
df_matches = pd.read_csv("./data/matches_info_SL_cleaned.csv")
df_lineups = pd.read_csv("./data/lineups_2022_2023_SL_cleaned.csv")


Adding playing minutes for each player

In [41]:
import pandas as pd

# Load the datasets
df_lineups = pd.read_csv('./data/lineups_2022_2023_SL_cleaned.csv')
events_df = pd.read_csv('./data/match_events_2022_2023_SL.csv')

# Initialize 'Minutes Played' based on whether the player is in the starting lineup or not
df_lineups['Minutes Played'] = df_lineups['Status'].apply(lambda status: 90 if status == 'Starting' else 0)

# Filter the events DataFrame to only include substitution events
substitutions_df = events_df[events_df['Event'] == 'Substitution']

# Update 'Minutes Played' for players substituted out
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_out = substitution['Player Out']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted out
    df_lineups.loc[(df_lineups['Match ID'] == match_id) & (df_lineups['Player'] == player_out), 'Minutes Played'] = timestamp

# Update 'Minutes Played' for players substituted in
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_in = substitution['Player Event']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted in
    df_lineups.loc[(df_lineups['Match ID'] == match_id) & (df_lineups['Player'] == player_in), 'Minutes Played'] = 90 - timestamp

# Create a new 'Player Event' column based on the 'Status' and 'Minutes Played' columns
df_lineups['Player Event'] = ''
df_lineups.loc[(df_lineups['Status'] == 'Starting') & (df_lineups['Minutes Played'] != 90), 'Player Event'] = 'Substituted Out'
df_lineups.loc[(df_lineups['Status'] == 'Substitute') & (df_lineups['Minutes Played'] != 0), 'Player Event'] = 'Substituted In'

# Save the updated dataframe to a new CSV
df_lineups.to_csv('./data/lineups_2022_2023_SL_cleaned.csv', index=False)


In [49]:
# Load the dataset
df = pd.read_csv('./data/lineups_2022_2023_SL_cleaned.csv')

# Define a function to convert the values to millions
def convert_to_millions(value):
    # Ensure the value is a string
    value = str(value)
    # Remove the currency symbol and any potential whitespace
    value = value.replace('€', '').replace('â‚¬', '').strip()
    # Check for the 'm' or 'k' suffix and convert accordingly
    if value.endswith('m'):
        # Convert from 'm' to float and strip the suffix
        return float(value[:-1])
    elif value.endswith('k'):
        # Convert from 'k' to millions and strip the suffix
        return float(value[:-1]) / 1000
    elif value == '' or pd.isnull(value):
        # If the value is blank or NaN, return 0
        return 0
    else:
        # If there's no suffix, just convert to float
        return float(value)

# Apply the conversion to the 'Market Value' column
df['Market Value'] = df['Market Value'].apply(lambda x: convert_to_millions(x))

# Convert the column to a numeric type, filling in NaNs with 0
df['Market Value'] = pd.to_numeric(df['Market Value'], errors='coerce').fillna(0)


# Save the modified dataset
df.to_csv('./data/lineups_2022_2023_SL_cleaned.csv', index=False)

In [35]:
import mysql.connector
import pandas as pd
from datetime import datetime

# Replace these placeholders with your MySQL credentials
db_config = {
    "host": "127.0.0.1",
    "user": "pbl_ra_mh",
    "password": "pblservette2024",
    "database": "transfermarkt"
}

# Establish a connection to the MySQL server
connection = mysql.connector.connect(**db_config)

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# Assuming you have loaded your CSV data into two pandas DataFrames:
# df_matches for "matches_info_SL_cleaned"
# df_lineups for "lineups_2022_2023_SL_cleaned"

# Create the table for matches
create_table_matches_query = """
CREATE TABLE IF NOT EXISTS `matches_info_SL_cleaned` (
    `MatchID` INT,
    `HomeTeam` VARCHAR(255),
    `HomeTeamPoints` INT,
    `AwayTeamPoints` INT,
    `AwayTeam` VARCHAR(255),
    `Result` VARCHAR(10),
    `League` VARCHAR(255),
    `ResultEnd` VARCHAR(5),
    `ResultHT` VARCHAR(5),
    `GoalsHomeTeam` INT,
    `GoalsAwayTeam` INT
)
"""
cursor.execute(create_table_matches_query)

# Insert data into matches table
for _, row in df_matches.iterrows():
    insert_matches_query = """
    INSERT INTO `matches_info_SL_cleaned` (`MatchID`, `HomeTeam`, `HomeTeamPoints`, `AwayTeamPoints`, `AwayTeam`, `Result`, `League`, `ResultEnd`, `ResultHT`, `GoalsHomeTeam`, `GoalsAwayTeam`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values_matches = (
        row['Match ID'],
        row['Home Team'],
        row['HomeTeamPoints'],
        row['AwayTeamPoints'],
        row['Away Team'],
        row['Result'],
        row['League'],
        row['result_end'],
        row['result_ht'],
        row['goals_HomeTeam'],
        row['goals_AwayTeam']
    )
    cursor.execute(insert_matches_query, values_matches)

# Create the table for lineups
create_table_lineups_query = """
CREATE TABLE IF NOT EXISTS `lineups_2022_2023_SL_cleaned` (
    `Position` VARCHAR(50),
    `Player` VARCHAR(255),
    `Age` INT,
    `MarketValue` INT,
    `Club` VARCHAR(255),
    `Gameday` VARCHAR(50),
    `HomeAway` VARCHAR(4),
    `Status` VARCHAR(10),
    `MatchID` INT,
    `Matchday` VARCHAR(50),
    `Weekday` VARCHAR(3),
    `Date` DATE,
    `Time` TIME,
)
"""
cursor.execute(create_table_lineups_query)

# Convert the 'date' column to datetime format if it hasn't been converted already
df_lineups['date'] = pd.to_datetime(df_lineups['date'], format='%d/%m/%Y')

# Insert data into lineups table
for _, row in df_lineups.iterrows():
    # Handle NaN or None values and convert the Market Value to millions with decimal places
    market_value_str = str(row['Market Value']).replace('€', '')
    market_value = 0.0
    if 'm' in market_value_str:
        market_value = float(market_value_str.replace('m', ''))
    elif 'k' in market_value_str:
        market_value = float(market_value_str.replace('k', '')) / 1000

    # Convert the date
    date_value = row['date'].strftime('%Y-%m-%d')  # Format the date as MySQL expects it

    # Convert the 'time' to 24-hour format if it's not already
    time_value = pd.to_datetime(row['time'], format='%I:%M %p').strftime('%H:%M:%S')

    insert_lineups_query = """
    INSERT INTO `lineups_2022_2023_SL_cleaned` (
        `Position`, `Player`, `Age`, `MarketValue`, `Club`, `Gameday`, `HomeAway`, 
        `Status`, `MatchID`, `Matchday`, `Weekday`, `Date`, `Time`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values_lineups = (
        row['Position'],
        row['Player'],
        row['Age'],
        market_value,  # This is already a float with the value in millions
        row['Club'],
        row['Gameday'],
        row['H/A'],
        row['Status'],
        row['Match ID'],
        row['matchday'],
        row['weekday'],
        date_value,
        time_value,
    )
    # Execute the insert query only if the market value is not None
    if market_value is not None:
        cursor.execute(insert_lineups_query, values_lineups)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

# Print a success message
print("Data insertion into MySQL completed successfully")


Data insertion into MySQL completed successfully


In [6]:


df_lineups = pd.read_csv("./data/lineups_2022_2023_SL_cleaned.csv")

# Replace NaN values in the 'Player Event' column with None
df_lineups['Player Event'] = df_lineups['Player Event'].where(pd.notnull(df_lineups['Player Event']), None)


In [8]:
import mysql.connector
import pandas as pd
from datetime import datetime

# Replace these placeholders with your MySQL credentials
db_config = {
    "host": "127.0.0.1",
    "user": "pbl_ra_mh",
    "password": "pblservette2024",
    "database": "transfermarkt"
}

# Establish a connection to the MySQL server
connection = mysql.connector.connect(**db_config)

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# Load your data into a pandas DataFrame
# df_lineups = pd.read_csv('path_to_your_csv.csv') # Uncomment and provide the correct path to your CSV

# Create the table for lineups
create_table_lineups_query = """
CREATE TABLE IF NOT EXISTS `lineups_2022_2023_SL_complete` (
    `Position` VARCHAR(50),
    `Player` VARCHAR(255),
    `Age` INT,
    `MarketValue` FLOAT,
    `Club` VARCHAR(255),
    `Gameday` VARCHAR(50),
    `HomeAway` VARCHAR(4),
    `Status` VARCHAR(10),
    `MatchID` INT,
    `Matchday` VARCHAR(50),
    `Weekday` VARCHAR(3),
    `Date` DATE,
    `Time` TIME,
    `MinutesPlayed` INT,
    `PlayerEvent` VARCHAR(50)
)
"""
cursor.execute(create_table_lineups_query)

# Convert the 'date' column to datetime format if it hasn't been converted already
df_lineups['date'] = pd.to_datetime(df_lineups['date'], format='%d/%m/%Y')

# Insert data into lineups table
for _, row in df_lineups.iterrows():

    # Convert the date
    date_value = row['date'].strftime('%Y-%m-%d')  # Format the date as MySQL expects it

    # Convert the 'time' to 24-hour format if it's not already
    time_value = pd.to_datetime(row['time'], format='%I:%M %p').strftime('%H:%M:%S')

    insert_lineups_query = """
    INSERT INTO `lineups_2022_2023_SL_complete` (
        `Position`, `Player`, `Age`, `MarketValue`, `Club`, `Gameday`, `HomeAway`, 
        `Status`, `MatchID`, `Matchday`, `Weekday`, `Date`, `Time`, `MinutesPlayed`, `PlayerEvent`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values_lineups = (
        row['Position'],
        row['Player'],
        row['Age'],
        row['Market Value'],  # Make sure this column name matches your DataFrame exactly
        row['Club'],
        row['Gameday'],
        row['H/A'],          # Make sure this column name matches your DataFrame exactly
        row['Status'],
        row['Match ID'],     # Make sure this column name matches your DataFrame exactly
        row['matchday'],     # Make sure this column name matches your DataFrame exactly
        row['weekday'],      # Make sure this column name matches your DataFrame exactly
        date_value,
        time_value,
        row['Minutes Played'],  # Make sure this column name matches your DataFrame exactly
        row['Player Event']     # Make sure this column name matches your DataFrame exactly
    )
    cursor.execute(insert_lineups_query, values_lineups)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

# Print a success message
print("Data insertion into MySQL completed successfully")


Data insertion into MySQL completed successfully


In [39]:
import pandas as pd

# Load the datasets
lineups_df = pd.read_csv('./data/lineups_2022_2023_SL_cleaned.csv')
events_df = pd.read_csv('./data/match_events_2022_2023_SL.csv')

# Initialize 'Minutes Played' based on whether the player is in the starting lineup or not
lineups_df['Minutes Played'] = lineups_df['Status'].apply(lambda status: 90 if status == 'Starting' else 0)

# Filter the events DataFrame to only include substitution events
substitutions_df = events_df[events_df['Event'] == 'Substitution']

# Update 'Minutes Played' for players substituted out
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_out = substitution['Player Out']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted out
    lineups_df.loc[(lineups_df['Match ID'] == match_id) & (lineups_df['Player'] == player_out), 'Minutes Played'] = timestamp

# Update 'Minutes Played' for players substituted in
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_in = substitution['Player Event']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted in
    lineups_df.loc[(lineups_df['Match ID'] == match_id) & (lineups_df['Player'] == player_in), 'Minutes Played'] = 90 - timestamp

# Create a new 'Player Event' column based on the 'Status' and 'Minutes Played' columns
lineups_df['Player Event'] = ''
lineups_df.loc[(lineups_df['Status'] == 'Starting') & (lineups_df['Minutes Played'] != 90), 'Player Event'] = 'Substituted Out'
lineups_df.loc[(lineups_df['Status'] == 'Substitute') & (lineups_df['Minutes Played'] != 0), 'Player Event'] = 'Substituted In'

# Save the updated dataframe to a new CSV
lineups_df.to_csv('./data/updated_lineups_with_minutes_and_events.csv', index=False)
